In [1]:
# import libraries

#Plotting 
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import time
import ta
import datetime as dt

#Data Processing
from pyspark.sql import *
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from matplotlib import pyplot as plt
from pylab import rcParams

# Spark.ml
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml import Pipeline
from sklearn.model_selection import cross_val_predict
import xgboost as xgb
from xgboost import XGBRegressor

In [2]:
%sh /home/ubuntu/databricks/python/bin/pip install ta --pre

Collecting ta
 Downloading https://files.pythonhosted.org/packages/06/fc/72e063cb6c0580ed986dbada2e07a5689f8ac6aaa31fcd32cf7599f46708/ta-0.3.8.tar.gz
Requirement already satisfied: numpy in /databricks/python3/lib/python3.5/site-packages (from ta) (1.16.2)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.5/site-packages (from ta) (0.19.2)
Requirement already satisfied: python-dateutil>=2 in /databricks/python3/lib/python3.5/site-packages (from pandas->ta) (2.5.3)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.5/site-packages (from pandas->ta) (2016.6.1)
Requirement already satisfied: six>=1.5 in /usr/lib/python3/dist-packages (from python-dateutil>=2->pandas->ta) (1.10.0)
Building wheels for collected packages: ta
 Running setup.py bdist_wheel for ta: started
 Running setup.py bdist_wheel for ta: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/17/72/b3/b13075678d17dadc9b238b5abf7c09597afa0ae755e27e7d1b
Successfully built ta
Installing collected packages: ta
Successfully installed ta-0.3.8
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [3]:
# File location and type
file_location = "/FileStore/tables/AAPL__2_-1028f.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

Date,Open,High,Low,Close,Adj Close,Volume
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.023007,117258400
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.021807,43971200
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.020206,26432000
1980-12-17,0.462054,0.464286,0.462054,0.462054,0.020706,21610400
1980-12-18,0.475446,0.477679,0.475446,0.475446,0.021307,18362400
1980-12-19,0.504464,0.506696,0.504464,0.504464,0.022607,12157600
1980-12-22,0.529018,0.531250,0.529018,0.529018,0.023707,9340800
1980-12-23,0.551339,0.553571,0.551339,0.551339,0.024708,11737600
1980-12-24,0.580357,0.582589,0.580357,0.580357,0.026008,12000800
1980-12-26,0.633929,0.636161,0.633929,0.633929,0.028409,13893600


In [4]:
# File location and type
file_location = "/FileStore/tables/GSPC.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_sp = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_sp)

Date,Open,High,Low,Close,Adj Close,Volume
1980-12-12,127.360001,129.979996,127.150002,129.229996,129.229996,39530000
1980-12-15,129.229996,131.330002,128.639999,129.449997,129.449997,39700000
1980-12-16,129.449997,131.220001,128.330002,130.600006,130.600006,41630000
1980-12-17,130.600006,133.589996,130.220001,132.889999,132.889999,50800000
1980-12-18,132.889999,135.899994,131.889999,133.000000,133.000000,69570000
1980-12-19,133.000000,134.000000,131.800003,133.699997,133.699997,50770000
1980-12-22,133.699997,136.679993,132.880005,135.779999,135.779999,51950000
1980-12-23,135.779999,137.479996,134.009995,135.300003,135.300003,55260000
1980-12-24,135.300003,136.550003,134.149994,135.880005,135.880005,29490000
1980-12-26,135.880005,137.020004,135.199997,136.570007,136.570007,16130000


In [5]:
# join two datasets together
df = df.join(df_sp, df.Date == df_sp.Date).select(df.High,df.Volume,df.Close,df.Date,df.Low,df_sp.Close.alias('sp'))
display(df)

High,Volume,Close,Date,Low,sp
0.515625,117258400,0.513393,1980-12-12,0.513393,129.229996
0.488839,43971200,0.486607,1980-12-15,0.486607,129.449997
0.453125,26432000,0.450893,1980-12-16,0.450893,130.600006
0.464286,21610400,0.462054,1980-12-17,0.462054,132.889999
0.477679,18362400,0.475446,1980-12-18,0.475446,133.000000
0.506696,12157600,0.504464,1980-12-19,0.504464,133.699997
0.531250,9340800,0.529018,1980-12-22,0.529018,135.779999
0.553571,11737600,0.551339,1980-12-23,0.551339,135.300003
0.582589,12000800,0.580357,1980-12-24,0.580357,135.880005
0.636161,13893600,0.633929,1980-12-26,0.633929,136.570007


In [6]:
# Select dataframe for future use, name it as df2
df1 = df.select(df.Date.cast("timestamp"),df.Close.cast("float"),df.Volume.cast("integer"),df.Low.cast("float"),df.High.cast("float"),df.sp.cast("float"))
df2 = df1.toPandas()
display(df1)

Date,Close,Volume,Low,High,sp
1980-12-12T00:00:00.000+0000,0.513393,117258400,0.513393,0.515625,129.23
1980-12-15T00:00:00.000+0000,0.486607,43971200,0.486607,0.488839,129.45
1980-12-16T00:00:00.000+0000,0.450893,26432000,0.450893,0.453125,130.6
1980-12-17T00:00:00.000+0000,0.462054,21610400,0.462054,0.464286,132.89
1980-12-18T00:00:00.000+0000,0.475446,18362400,0.475446,0.477679,133.0
1980-12-19T00:00:00.000+0000,0.504464,12157600,0.504464,0.506696,133.7
1980-12-22T00:00:00.000+0000,0.529018,9340800,0.529018,0.53125,135.78
1980-12-23T00:00:00.000+0000,0.551339,11737600,0.551339,0.553571,135.3
1980-12-24T00:00:00.000+0000,0.580357,12000800,0.580357,0.582589,135.88
1980-12-26T00:00:00.000+0000,0.633929,13893600,0.633929,0.636161,136.57


In [7]:
# Checking the historical stock price of AAPL
ax = df2.plot(x='Date', y='Close', style='b-', grid=True, linewidth = 2)
ax.set_xlabel("Date")
ax.set_ylabel("USD")
ax.set_title("Close Price of AAPL")
display()

In [8]:
# fill NA row
df2.iloc[165,:] = df2.iloc[164,:]
df2.dropna(inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9391 entries, 0 to 9390
Data columns (total 6 columns):
Date 9391 non-null datetime64[ns]
Close 9391 non-null float32
Volume 9391 non-null float64
Low 9391 non-null float32
High 9391 non-null float32
sp 9391 non-null float32
dtypes: datetime64[ns](1), float32(4), float64(1)
memory usage: 366.8 KB

In [9]:
# Prepare features of Gradient Boosting

features = pd.DataFrame(df2["Close"].rolling(window=2).mean())
features.columns = ['sma_2']
features['sma_5'] = pd.DataFrame(df2["Close"].rolling(window=5).mean())
features['sma_10'] = pd.DataFrame(df2["Close"].rolling(window=10).mean())
features['26_ema'] = pd.ewma(df2["Close"], span=26)
features['12_ema'] = pd.ewma(df2["Close"], span=12)
features['MACD'] = features['12_ema'] - features['26_ema']
features['pct_change'] = df2.Volume.pct_change(periods=4)
features['zscore'] = pd.DataFrame((df2['Close']-df2['Close'].rolling(window=26).mean())/df2['Close'].rolling(window=26).std())
features['percentile'] = pd.DataFrame(df2.Volume.rolling(26,min_periods=20).apply(lambda x: pd.Series(x).rank(pct=True)[0]))
features['lag_1'] = df2['Close'].shift(1)
features['lag_5'] = df2['Close'].shift(5)
features['loss'] = df2['Close'] - df2['Close'].shift(-1)
features['loss_5'] = df2['Close'] - df2['Close'].shift(-5)
features['momentum']= ta.momentum.money_flow_index(df2.High, df2.Low, df2.Close, df2.Volume, n=10, fillna=False)
features['momentum2'] = features['momentum'] - features['momentum'].rolling(window=26).mean()
features['sign'] = features['pct_change'].apply(np.sign)
features['plus_minus'] = features['sign'].rolling(window=20).sum()
features['Date'] = df2['Date']
features['dayofweek'] = features['Date'].dt.dayofweek
features['quarter'] = features['Date'].dt.quarter
features['month'] = features['Date'].dt.month
features['year'] = features['Date'].dt.year
features['dayofyear'] = features['Date'].dt.dayofyear
features['dayofmonth'] = features['Date'].dt.day
features['weekofyear'] = features['Date'].dt.weekofyear
features['sp'] = df2['sp']
features = features.dropna()
features['Close'] = df2.Close
df4=features
df4['prediction_1D'] = df4['Close'].shift(-1)
df4['prediction_1W'] = df4['Close'].shift(-5)
df4['prediction_1M'] = df4['Close'].shift(-26)
df4['prediction_4M'] = df4['Close'].shift(-104)
df4= df4.dropna()
df4.info()

/local_disk0/tmp/1554340712398-0/PythonShell.py:8: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,span=26,min_periods=0,ignore_na=False).mean()
 import signal
/local_disk0/tmp/1554340712398-0/PythonShell.py:9: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,span=12,min_periods=0,ignore_na=False).mean()
 import six.moves.queue
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9248 entries, 34 to 9281
Data columns (total 31 columns):
sma_2 9248 non-null float64
sma_5 9248 non-null float64
sma_10 9248 non-null float64
26_ema 9248 non-null float64
12_ema 9248 non-null float64
MACD 9248 non-null float64
pct_change 9248 non-null float64
zscore 9248 non-null float64
percentile 9248 non-null float64
lag_1 9248 non-null float32
lag_5 9248 non-null float32
loss 9248 non-null float32
loss_5 9248 non-null float32
momentum 9248 non-null float64
momentum2 9248 non-null float64
sign 9248 non-null float64
plus_minus 9248 non-null float64
Date 9248 non-null datetime64[ns]
dayofweek 9248 non-null int64
quarter 9248 non-null int64
month 9248 non-null int64
year 9248 non-null int64
dayofyear 9248 non-null int64
dayofmonth 9248 non-null int64
weekofyear 9248 non-null int64
sp 9248 non-null float32
Close 9248 non-null float32
prediction_1D 9248 non-null float32
prediction_1W 9248 non-null float32
prediction_1M 9248 non-null float32
prediction_4M 9248 non-null float32
dtypes: datetime64[ns](1), float32(10), float64(13), int64(7)
memory usage: 1.9 MB

In [10]:
# Prepare training data and validation data
train_xgb = df4[:8000]
valid_xgb = df4[8000:]

x_train = train_xgb[['sma_2', 'sma_5', 'sma_10', '26_ema', '12_ema', 'MACD', 'pct_change',
       'zscore', 'percentile', 'lag_1', 'lag_5', 'loss', 'loss_5', 'momentum',
       'momentum2', 'sign', 'plus_minus', 'dayofweek', 'quarter',
       'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'sp', 'Close']]

y_train = train_xgb['prediction_4M']

x_test = valid_xgb[['sma_2', 'sma_5', 'sma_10', '26_ema', '12_ema', 'MACD', 'pct_change',
       'zscore', 'percentile', 'lag_1', 'lag_5', 'loss', 'loss_5', 'momentum',
       'momentum2', 'sign', 'plus_minus', 'dayofweek', 'quarter',
       'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'sp', 'Close']]

y_test = valid_xgb['prediction_4M']

In [11]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)


In [12]:
# Fit the model with x_train and y_train
xgb_grid.fit(x_train,y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done 18 out of 18 | elapsed: 6.1min finished
 Out[ 128 ]: 
GridSearchCV(cv=2, error_score='raise-deprecating',
 estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
 colsample_bytree=1, gamma=0, importance_type='gain',
 learning_rate=0.1, max_delta_step=0, max_depth=3,
 min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
 nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
 reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
 subsample=1),
 fit_params=None, iid='warn', n_jobs=5,
 param_grid={'min_child_weight': [4], 'max_depth': [5, 6, 7], 'n_estimators': [500], 'objective': ['reg:linear'], 'subsample': [0.7], 'colsample_bytree': [0.7], 'silent': [1], 'nthread': [4], 'learning_rate': [0.03, 0.05, 0.07]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
 scoring=None, verbose=True)

In [13]:
#Print out the best parameters for the model
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

-0.21425309091802286
{'nthread': 4, 'max_depth': 5, 'n_estimators': 500, 'objective': 'reg:linear', 'subsample': 0.7, 'colsample_bytree': 0.7, 'silent': 1, 'min_child_weight': 4, 'learning_rate': 0.03}

In [14]:
#Train the model with the best parameters for the model
reg = XGBRegressor(nthread=4, max_depth=5, n_estimators=500, objective = 'reg:linear', subsample = 0.7, colsample_bytree = 0.7, silent = 1, min_child_weight = 4,learning_rate = 0.03)

In [15]:
#Find out the predictions for time horizon of 1 day
preds_1D=cross_val_predict(reg, x_test, y_test, cv=10)

In [16]:
#Find out the predictions for time horizon of 1 week
preds_1W=cross_val_predict(reg, x_test, y_test, cv=10)

In [17]:
#Find out the predictions for time horizon of 1 month
preds_1M=cross_val_predict(reg, x_test, y_test, cv=10)

In [18]:
#Find out the predictions for time horizon of 4 months
preds_4M=cross_val_predict(reg, x_test, y_test, cv=10)

In [19]:
# Calculate the SMAPE score for time horizon of 1 day
preds = preds_1D
y_valid = y_test
SMAPE_1D = 100/len(np.array(y_valid)) * np.sum(2 * np.abs(np.array(preds) - np.array(y_valid)) / (np.abs(np.array(y_valid)) + np.abs(np.array(preds))))
SMAPE_1D

Out[ 111 ]: 1.2702905978911962

In [20]:
# Calculate the SMAPE for time horizon of 1 week
preds = preds_1W
y_valid = y_test
SMAPE_1W = 100/len(np.array(y_valid)) * np.sum(2 * np.abs(np.array(preds) - np.array(y_valid)) / (np.abs(np.array(y_valid)) + np.abs(np.array(preds))))
SMAPE_1W

Out[ 120 ]: 2.372157268035106

In [21]:
# Calculate the SMAPE for time horizon of 1 month
preds = preds_1M
y_valid = y_test
SMAPE_1M = 100/len(np.array(y_valid)) * np.sum(2 * np.abs(np.array(preds) - np.array(y_valid)) / (np.abs(np.array(y_valid)) + np.abs(np.array(preds))))
SMAPE_1M

Out[ 126 ]: 7.121055554120969

In [22]:
# Calculate the SMAPE for time horizon of 4 months
preds = preds_4M
y_valid = y_test
SMAPE_4M = 100/len(np.array(y_valid)) * np.sum(2 * np.abs(np.array(preds) - np.array(y_valid)) / (np.abs(np.array(y_valid)) + np.abs(np.array(preds))))
SMAPE_4M

Out[ 132 ]: 9.229148351229155

In [23]:
# Create a table with all the predictions and actual closing price
list_of_tuples = list(zip(df4[8000:].Date,preds_1D,preds_1W,preds_1M,preds_4M,df4[8000:].Close))

df_display = pd.DataFrame(list_of_tuples,columns = ['Date','Prediction_1D','Prediction_1W','Prediction_1M','Prediction_4M','Close'])
df_display_spark = sqlContext.createDataFrame(df_display)
display(df_display_spark.take(2000))

Date,Prediction_1D,Prediction_1W,Prediction_1M,Prediction_4M,Close
2012-10-16T00:00:00.000+0000,91.40980529785156,88.26142883300781,93.33838653564453,91.74507904052734,92.82714080810547
2012-10-17T00:00:00.000+0000,90.89107513427734,89.86825561523438,94.2605972290039,92.6423568725586,92.08714294433594
2012-10-18T00:00:00.000+0000,88.43246459960938,89.87409973144531,92.31644439697266,93.05110168457031,90.37714385986328
2012-10-19T00:00:00.000+0000,92.15917205810547,91.40338897705078,90.15206909179688,92.55355072021484,87.12000274658203
2012-10-22T00:00:00.000+0000,87.72341918945312,87.76653289794922,90.96519470214844,91.09017181396484,90.57571411132812
2012-10-23T00:00:00.000+0000,89.93582153320312,90.68633270263672,92.4299545288086,92.41841125488281,87.62285614013672
2012-10-24T00:00:00.000+0000,88.10498046875,86.61795806884766,91.3186264038086,90.45143127441406,88.11856842041016
2012-10-25T00:00:00.000+0000,88.26215362548828,88.48250579833984,89.81781005859375,91.41162872314453,87.07714080810547
2012-10-26T00:00:00.000+0000,86.78085327148438,88.39395141601562,88.47952270507812,86.38508605957031,86.28571319580078
2012-10-31T00:00:00.000+0000,87.08246612548828,84.96488952636719,88.94287109375,87.02201843261719,85.04571533203125


In [24]:
%sh /home/ubuntu/databricks/python/bin/pip install xgboost --pre

Collecting xgboost
 Downloading https://files.pythonhosted.org/packages/6a/49/7e10686647f741bd9c8918b0decdb94135b542fe372ca1100739b8529503/xgboost-0.82-py2.py3-none-manylinux1_x86_64.whl (114.0MB)
Requirement already satisfied: scipy in /databricks/python3/lib/python3.5/site-packages (from xgboost) (1.2.1)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.5/site-packages (from xgboost) (1.16.2)
Installing collected packages: xgboost
Successfully installed xgboost-0.82
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [25]:
xgb.plot_importance(reg, height=0.9)
display()